<a href="https://colab.research.google.com/github/drod75/Weeaboo-Buddy/blob/main/notebooks/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Package installation

In [ ]:
!pip install -U langchain-community tavily-python python-dotenv langchain-google-genai langgraph langchain-tavily

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI

## Testing

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if t["name"] not in self.tools:  # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from google.colab import userdata

import os

tkey = userdata.get("TAVILY_API_KEY")
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = tkey

gkey = userdata.get("GOOGLE_API_KEY")
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = gkey

tool = TavilySearchResults(max_results=4)

In [ ]:
query = (
    "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
)
messages = [HumanMessage(content=query)]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(
    model,
    [tool],
    system="You are an American football expert who does research of every team and there stats each season.",
)
result = abot.graph.invoke({"messages": messages})

print(result["messages"][-1].content)

## Testing 2

In [ ]:
prompt = """
You are a crunchyroll data analyst who specialzes in researching various different types of anime to help questions for fans,
as well as other analysts who specialize in drawing up graph and statistics.
"""
query = "Who is Zoro from One Piece"

messages = [HumanMessage(content=query)]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

print(result["messages"][-1].content)

In [ ]:
! pip install jikanpy-v4

In [ ]:
from jikanpy import Jikan

jikan = Jikan()

bebop = jikan.anime(1)
bebop

In [ ]:
current_season = jikan.seasons(extension="now")
current_season

In [ ]:
search_result = jikan.search("anime", "Boku no Piko")
search_result

In [ ]:
from langchain_core.tools import tool


@tool
def search_anime(query: str) -> dict:
    """
    Name:
      search_anime
    Description:
      Search for an anime by a name, the result will be a json of possile solutions, from best to worst match.
    Args:
      query: anime name
    """

    search_result = jikan.search("anime", query)
    return search_result


@tool
def get_stats_by_id(id: int) -> dict:
    """
    Name:
      get_stats_by_id
    Description:
      Get the stats of an anime by it's id
    Args:
      id: anime id
    """

    result = jikan.anime(id)
    return result


@tool
def get_seasonal() -> dict:
    """
    Name:
      get_seasonal
    Description:
      Get the anime from this season
    Args:
      None
    """
    current_season = jikan.seasons(extension="now")
    return current_season


tavily = TavilySearchResults(max_results=4)
tools = [search_anime, get_stats_by_id, get_seasonal, tavily]

In [ ]:
prompt = """
You are a crunchyroll data analyst who specialzes in researching various different types of anime to help questions for fans,
as well as other analysts who specialize in drawing up graph and statistics.
"""
query = "What are the anime from this current season?"

messages = [HumanMessage(content=query)]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(model, tools, system=prompt)
result = abot.graph.invoke({"messages": messages})

print(result["messages"][-1].content)